# Welcome to a visual simulator of the fifth problem!


*   In this Simulator you can simulate your fit_shape + area problems, to see how it works using graphing libraries
*   Insert the AbstractShape class alongside your own MyShape and Assignment5 classes and see how well your functions work
*   Make sure your MyShape class has the following methods: sample, area, contour (Like AbstractShape)
*This simulator is an upgraded version of the tests that can be run in the assignment
*Please download a copy of this script so that the original won't be altered



##Import the necessary libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
import random
from scipy.spatial import distance_matrix, ConvexHull
from scipy.sparse.csgraph import minimum_spanning_tree


##The AbstractShape Class



In [ ]:
class AbstractShape:
    """
    An abstract class that represents a closed shape.
    """

    def __init__(self):
        pass

    def sample(self):
        """
        Returns
        -------
        A random point (x,y) on the shape contour.

        """
        return (0, 0)

    def contour(self, n: int):
        """
        This method is used to draw the shape contour. It returns an array of
        consecutive points on the shape contour. Larger n results in smoother
        shapes.

        Parameters
        ----------
        n : int
            the number of points on the shape contour to return.

        Returns
        -------
        np.ndarray((n,2))
            An array of consecutive points on the shape contour.

        """
        return np.ndarray((n, 2), dtype=np.float32)

    def area(self) -> np.float32:
        """

        Returns
        -------
        float
            The area of the shape.

        """
        return np.float32(1.0)

##Insert the MyShape class here if needed⬇


In [ ]:
class MyShape(AbstractShape):
    # change this class with anything you need to implement the shape
    def __init__(self):
        super(MyShape, self).__init__()
        pass

##Run this, these are some polygon classes + shapes from the original files


In [ ]:
class Circle(AbstractShape):
    def __init__(self, cx: np.float32, cy: np.float32, radius: np.float32, noise: np.float32):
        super().__init__()
        self._radius = radius
        self._noise = noise
        self._cx = cx
        self._cy = cy

    def sample(self):
        w = np.random.random() * 2 * np.pi
        x = np.cos(w) * self._radius + self._cx
        x += np.random.randn() * self._noise
        y = np.sin(w) * self._radius + self._cy
        y += np.random.randn() * self._noise
        return x, y

    def contour(self, n: int):
        w = np.linspace(0, 2 * np.pi, num=n)
        x = np.cos(w) * self._radius + self._cx
        y = np.sin(w) * self._radius + self._cy
        xy = np.stack((x, y), axis=1)
        return xy

    def area(self):
        a = np.pi * self._radius ** 2
        return a

def bezier3(P1, P2, P3, P4):
    M = np.array(
        [[-1, +3, -3, +1],
         [+3, -6, +3, 0],
         [-3, +3, 0, 0],
         [+1, 0, 0, 0]],
        dtype=np.float32
    )
    P = np.array([P1, P2, P3, P4], dtype=np.float32)

    def f(t):
        T = np.array([t ** 3, t ** 2, t, 1], dtype=np.float32)
        return T.dot(M).dot(P)

    return f
class BezierShape(AbstractShape):
    def __init__(self, knots, control, noise):
        super().__init__()
        self.knots = knots
        self._control = control
        self._noise = noise
        self._n = len(knots)

        self._fs = [
            bezier3(knots[i - 1], control[2 * i], control[2 * i + 1], knots[i])
            for i in range(self._n)
        ]

    def sample(self):
        i = np.random.randint(self._n)
        t = np.random.random()
        x, y = self._fs[i](t)
        x += np.random.randn() * self._noise
        y += np.random.randn() * self._noise
        return x, y

    def contour(self, n: int):
        ppf = n // self._n
        rem = n % self._n
        points = []
        for i in range(self._n):
            ts = np.linspace(0, 1, num=(ppf + 2 if i < rem else ppf + 1))
            for t in ts[0:-1]:
                x, y = self._fs[i](t)
                xy = np.array((x, y))
                points.append(xy)
        points = np.stack(points, axis=0)
        return points

    def area(self):
        a = 0
        cntr = self.contour(10000)
        for i in range(10000):
            x1, y1 = cntr[i - 1]
            x2, y2 = cntr[i]
            if x1 != x2:
                a += 0.5 * (x2 - x1) * (y1 + y2)
        return a


class Polygon(AbstractShape):
    def __init__(self, knots, noise):
        super().__init__()
        self.knots = knots
        self._noise = noise
        self._n = len(knots)

    def sample(self):
        i = np.random.randint(self._n)
        t = np.random.random()

        x1, y1 = self.knots[i - 1]
        x2, y2 = self.knots[i]

        x = np.random.random() * (x2 - x1) + x1
        x += np.random.randn() * self._noise

        y = np.random.random() * (y2 - y1) + y1
        y += np.random.randn() * self._noise
        return x, y

    def contour(self, n: int):
        ppf = n // self._n
        rem = n % self._n
        points = []
        for i in range(self._n):
            ts = np.linspace(0, 1, num=(ppf + 2 if i < rem else ppf + 1))

            x1, y1 = self.knots[i - 1]
            x2, y2 = self.knots[i]

            for t in ts[0:-1]:
                x = t * (x2 - x1) + x1
                y = t * (y2 - y1) + y1
                xy = np.array((x, y))
                points.append(xy)
        points = np.stack(points, axis=0)
        return points

    def area(self):
        a = 0
        for i in range(self._n):
            x1, y1 = self.knots[i - 1]
            x2, y2 = self.knots[i]
            a += 0.5 * (x2 - x1) * (y1 + y2)
        return a

def shape1() -> AbstractShape:
    return Polygon(
        knots=[(0, 0), (1, 1), (1, -1)],
        noise=0.1
    )


def shape2() -> AbstractShape:
    return Polygon(
        knots=[(0, 0), (0, 1), (1, 1), (1, 0)],
        noise=0.1
    )


def shape3() -> AbstractShape:
    return Polygon(
        knots=[(0, 0), (0, 8), (20, 1), (1, 0)],
        noise=0.3
    )


def shape4() -> AbstractShape:
    return Polygon(
        knots=[(0, 0), (0.5, 100000), (50000, 70000), (100000, 1), (50000, 0)],
        noise=2000
    )


def shape5() -> AbstractShape:
    return Polygon(
        knots=[(0, 0), (0.5, 100000), (1, 1), (100000, 1), (50000, 0)],
        noise=1.0
    )


def shape6() -> AbstractShape:
    return BezierShape(
        knots=[[0, 0], [0, 1]],
        control=[[2, 0.5], [2, 0.5], [-2, 0.5], [-2, 0.5]],
        noise=0.1
    )


def shape7() -> AbstractShape:
    return BezierShape(
        knots=[(0, 0), (1, 1), (1, -1)],
        control=[(0.2, 8), (0.8, 6), (3, 0.5), (-1, 0.5), (3, -5), (0.2, -0.6)],
        noise=0.1
    )

##Insert your Assignment5 class here ⬇


In [ ]:
class Assignment5:
    def __init__(self):
        """
        Here goes any one time calculation that need to be made before
        solving the assignment for specific functions.
        """

        pass

    def area(self, contour: callable, maxerr=0.001)->np.float32:
        """
        Compute the area of the shape with the given contour.

        Parameters
        ----------
        contour : callable
            Same as AbstractShape.contour
        maxerr : TYPE, optional
            The target error of the area computation. The default is 0.001.

        Returns
        -------
        The area of the shape.

        """
        return np.float32(1.0)

    def fit_shape(self, sample: callable, maxtime: float) -> AbstractShape:
        """
        Build a function that accurately fits the noisy data points sampled from
        some closed shape.

        Parameters
        ----------
        sample : callable.
            An iterable which returns a data point that is near the shape contour.
        maxtime : float
            This function returns after at most maxtime seconds.

        Returns
        -------
        An object extending AbstractShape.
        """

        # replace these lines with your solution
        result = MyShape()
        x, y = sample()

        return result

##Run this to see the circle shape test


In [ ]:
def noisy_circle(cx, cy, radius, noise): #noisy circle from original files
    return Circle(cx, cy, radius, noise)



cx, cy, radius, noise = 1, 1, 1, 0.1
circ = noisy_circle(cx=cx, cy=cy, radius=radius, noise=noise)
assign5 = Assignment5()


start_time = time.time()
shape = assign5.fit_shape(sample=circ.sample, maxtime=2)
elapsed_time = time.time() - start_time

calculated_area = assign5.area(circ.contour) #getting area from created area function

# Generate original circle points
theta = np.linspace(0, 2 * np.pi, 100)
original_circle_x = cx + radius * np.cos(theta)
original_circle_y = cy + radius * np.sin(theta)


true_area = np.pi * radius**2 #the actual area of the circle

plt.figure(figsize=(10, 10))
plt.title("Circle With Noise Visualisation")
plt.xlabel("X")
plt.ylabel("Y")




all_points = [circ.sample() for _ in range(700)]
all_x, all_y = zip(*all_points)
plt.scatter(all_x, all_y, color="lightgray", label="Noisy Points", s=10)

###########################################################
inliers = shape.contour(1000) #change this if needed
###########################################################

returned_area=shape.area()

if len(inliers) > 0:
    num_inliers = len(inliers)
    s = max(1500 // num_inliers, 60)
    inlier_x, inlier_y = zip(*inliers)
    plt.scatter(
        inlier_x,
        inlier_y,
        color="blue",
        label="Returned Points",
        s=s,
        edgecolor="black",
        alpha=0.8
    )

plt.plot(original_circle_x, original_circle_y, color="green", label="Original Circle", linestyle="--", linewidth=2)#plotting the original circle


plt.text(0.05, 0.95, f"Fitted Area: {calculated_area:.4f}\nTime: {elapsed_time:.2f}s\nTrue Area: {true_area:.4f}\nReturned Shape Area:{returned_area:.4f}", #printing relevant data in the plot
         transform=plt.gca().transAxes, fontsize=12,
         verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))


plt.legend()
plt.grid()
plt.axis("equal")
plt.show()

##Try tests from the grader file, change this according to the shape you want to test, numbers go from 1 to 7


In [ ]:
shape_instance = shape6() #to see other tests, change the number here, e.g shape4()

###Run this to see the results of your functions


In [ ]:



assign5 = Assignment5()#fitting the shape using your Assignment5 class
start_time = time.time()
fitted_shape = assign5.fit_shape(sample=shape_instance.sample, maxtime=1)
elapsed_time = time.time() - start_time


calculated_area = assign5.area(shape_instance.contour)#calculating the area of the fitted shape using implemented function

true_area =shape_instance.area()


plt.figure(figsize=(10, 10))
plt.xlabel("X")
plt.ylabel("Y")
plt.grid()
plt.axis("equal")


noisy_points = np.array([shape_instance.sample() for _ in range(700)])#plotting noisy points
plt.scatter(noisy_points[:, 0], noisy_points[:, 1], color="lightgray", label="Noisy Points", s=10)


true_contour = shape_instance.contour(1000) #plotting the original circle
true_contour = np.vstack([true_contour, true_contour[0]])
plt.plot(true_contour[:, 0], true_contour[:, 1], color="green", label="True Shape",linestyle="--", linewidth=2)

###########################################################
inliers = fitted_shape.contour(100) #change this if you need
###########################################################


returned_area=fitted_shape.area()
if len(inliers) > 0: #plotting the returned points
    inlier_x, inlier_y = zip(*inliers)
    num_inliers = len(inliers)
    s = max(1500 // num_inliers, 60)
    plt.scatter(
        inlier_x,
        inlier_y,
        color="blue",
        label="Returned Points",
        s=s,
        edgecolor="black",
        alpha=0.8
    )


plt.text(0.05, 0.95, f"Fitted Area: {calculated_area:.4f}\nTime: {elapsed_time:.2f}s\nTrue Area: {true_area:.4f}\nReturned Shape Area:{returned_area}",
         transform=plt.gca().transAxes, fontsize=12,
         verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))

plt.legend()
plt.show()

